In [1]:
import ee
from scipy.signal import savgol_filter
from src.controllers.time_series_hls import HLS
from src.controllers.plotter import PhenologyVisualizer
from src.controllers.metrics_vos_pos import VosPosMetrics
from src.controllers.metrics_bos_eso import BosEosMetrics
from src.controllers.geometry import ProcessadorGeoDataFrame

In [2]:
# Conecta ao Google Earth Engine
ee.Authenticate()
ee.Initialize()

In [3]:
# Datas de análise
start_date = '2022-12-01'
end_date = '2024-01-31'

#Paht polygons
path = r'data\sample_full.gpkg'

#Index polygon, defoult is 0
index_poligon = 5498

# Aplicar o filtro Savitzky-Golay
window_size = 30
poly_order = 4

#Order NDVI
order_ndvi = 30

In [4]:
#Read geometry file
processador = ProcessadorGeoDataFrame(path)
vertices, geometry = processador.extrair_coordenadas(index_poligon)

#Get the time sereis NDVI
hsl = HLS(geometry, start_date, end_date)
ndvi_df = hsl.convert_to_dataframe()

#Smooth time series with NDVI
ndvi_df['savitzky_golay'] = savgol_filter(ndvi_df['ndvi'], window_size, poly_order)

#Get and VOS and POS metrics
vos_pos_analyzer = VosPosMetrics(ndvi_df, order_ndvi)
phenology_df = vos_pos_analyzer.analyze_phenology()

#Get and BOS and EOS metrics
analysis = BosEosMetrics(ndvi_df, phenology_df, 0.457)
phenology_df  = analysis.execute_analysis()


#Plot the results
visualizer = PhenologyVisualizer(ndvi_df, phenology_df)
visualizer.convert_dates()
visualizer.create_plot()

c:\Users\ander\OneDrive\Área de Trabalho\msu_timeseries_ndvi\src\controllers\time_series_hls.py:125: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df = df.resample('D').interpolate(method='linear')
c:\Users\ander\OneDrive\Área de Trabalho\msu_timeseries_ndvi\src\controllers\metrics_bos_eso.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.interval_df['first_derivative'] = self.interval_df['savitzky_golay'].diff().fillna(0)


In [6]:
#Get and VOS and POS metrics
vos_pos_analyzer = VosPosMetrics(ndvi_df, order_ndvi)
phenology_df = vos_pos_analyzer.analyze_phenology()

ValueError: attempt to get argmax of an empty sequence